In [1]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-3qt598dv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-3qt598dv
  Resolved https://github.com/huggingface/transformers to commit 831bc25d8fdb85768402f772cf65cc3d7872b211
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8593748 sha256=998e1c0de7cc95a5ad361a51a6a31dc5f4bab584d3ed9b896e02ccc2234e1c9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-09o7jh71/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


In [3]:
!pip install datasets evaluate wandb torchmetrics torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00


In [4]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import GradScaler, autocast
from transformers import AutoModelForUniversalSegmentation, AutoProcessor
import torchvision.transforms as tvt
from torchvision.transforms.functional import crop
from torchmetrics.classification import Accuracy, MulticlassAccuracy, MulticlassJaccardIndex, JaccardIndex
import albumentations as A
from tqdm.auto import tqdm
from huggingface_hub import notebook_login
from datasets import load_dataset
import wandb
import evaluate
import statistics
from copy import deepcopy
from huggingface_hub import hf_hub_download
import torchinfo

import numpy as np
import pandas as pd
import random
import requests
import json
from pathlib import Path
import os
from typing import List, Dict, Tuple
import warnings
from PIL import Image as PILImage
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
#warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.empty_cache()
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = True

In [5]:
# import os
# os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [6]:
#!huggingface-cli login
from huggingface_hub import login
from google.colab import userdata
import wandb

login(token=userdata.get('HF_TOKEN'))
wandb.login(key=userdata.get('WANDB_API_KEY'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
def get_labels():
    data_directory = Path(".")
    json_file_path = data_directory / "labels_info.json"
    if json_file_path.is_file():
        print(f"[INFO]: Found {json_file_path}.Skipping Download...")
    else:
        print("[INFO]: Downloading labels_info.json from hub")
        json_file_path = hf_hub_download(
            repo_id="BhavanaMalla/railsem19-semantic-expanded",
            filename="labels_info.json",
            repo_type="dataset",
            local_dir=data_directory
        )
    with open(json_file_path, "r") as f:
        labels_info = json.load(f)
    id2label = labels_info["id2label"]
    label2id = labels_info["label2id"]
    labels = labels_info["labels"]
    color_palette = labels_info["color_palette"]

    # add the background label
    id2label["19"] = "background"
    label2id["background"] = 19
    labels.append("background")
    color_palette.append([0, 0, 0])

    # correcting the labels
    id2label = {int(key): value.replace('-', '_') \
                for key, value in id2label.items()}
    label2id = {key.replace('-', '_'): value for key, value in label2id.items()}
    labels = [label.replace('-', '_') for label in labels]
    return {"id2label": id2label, "label2id": label2id, "labels": labels,
            "color_palette": color_palette}

In [8]:
def load_railsem_dataset():
    print(f"[INFO]: Extracting Railsem19 dataset from hub on")
    railsem_ds = load_dataset("BhavanaMalla/railsem19-semantic-expanded")
    return railsem_ds

In [9]:
def get_modified_labels():
    modified_id2label = {
        0: "road", 1: "sidewalk", 2: "construction_fence",
        3: "rail_raised_rail_embedded", 4: "pole_traffic_light_traffic_sign",
        5: "sky", 6: "human", 7: "tram_track_rail_track", 8: "car_truck",
        9: "on_rails", 10: "vegetation", 11: "trackbed",
        12: "background_terrain"
    }
    modified_label2id = {
        label: id for id, label in modified_id2label.items()
    }
    modified_labels = [label for label in modified_id2label.values()]
    return modified_id2label, modified_label2id, modified_labels

In [10]:
class RemapBackground():
    """ Remap background to label 19 """
    def __call__(self, mask):
        return torch.where(mask > 18, 19, mask)


class RemapLabels():
    def __init__(self):
        self.class_mapping = get_labels()["id2label"]
        self.class_coding = {v: k for k, v in self.class_mapping.items()}

        # Remapping of original 20 labels to 13 labels
        self.modified_labels = {
            0: ["road"], 1: ["sidewalk"],
            2: ["construction", "fence"],
            3: ["rail_raised", "rail_embedded"],
            4: ["pole", "traffic_light", "traffic_sign"],
            5: ["sky"], 6: ["human"],
            7: ["tram_track", "rail_track"],
            8: ["car", "truck"], 9: ["on_rails"],
            10: ["vegetation"], 11: ["trackbed"],
            12: ["background", "terrain"]
        }
        self.modified_ids = {}
        for k, v in self.modified_labels.items():
            self.modified_ids[k] = [self.class_coding[label] for label in v]

    def __call__(self, mask):
        final_label = np.zeros_like(mask)
        for key, val in self.modified_ids.items():
            specific_label = np.zeros_like(mask)
            specific_label = np.where(np.isin(mask, np.array(val)), 1, 0)
            specific_label *= key
            final_label = np.add(final_label, specific_label)
        return torch.from_numpy(final_label)



In [11]:
class PatchQueue(Dataset):
    '''
    Extract random patches from an image -> put them in queue
    -> feed them to the dataloader
    ref : https://github.com/fepegar/torchio/blob/main/src/torchio/data/queue.py
    '''
    def __init__(self, dataset, max_length,  # Queue Length
                 samples_per_image,  # Patches per image
                 queue_workers=4,  # Num Workers
                 patch_size=[512, 512],  # Patch Size
                 shuffle=False):
        self.dataset = dataset
        self.max_length = max_length
        self.queue_workers = queue_workers
        self.samples_per_image = samples_per_image
        self.shuffle_queue = shuffle
        self.patch_size = patch_size
        self._images_iterable = None
        self.patch_list = list()
        self._num_sampled_images = 0
        self.steps_per_epoch = len(self.dataset) * self.samples_per_image
        self.resize_transform = tvt.Resize(patch_size[::-1])

    def __len__(self):
        return self.steps_per_epoch

    def __getitem__(self, item):
        if not self.patch_list:
            print("[WARN]: Patch List is empty...")
            self._fill()
            self.patch_list.reverse()
        sample_patch = self.patch_list.pop()
        return sample_patch

    @staticmethod
    def _get_first_item(batch):
        return batch[0]

    def initialize_images_iterable(self):
        self._images_iterable = self._get_images_iterable()

    @property
    def images_iterable(self):
        if self._images_iterable is None:
            self.initialize_images_iterable()
        return self.images_iterable

    def _get_images_iterable(self):
        if torch.distributed.is_available() and torch.distributed.is_initialized():
            sampler = torch.utils.data.DistributedSampler(
                self.dataset,
                num_replicas=torch.distributed.get_world_size(),
                rank=torch.distributed.get_rank(), shuffle=self.shuffle_queue
            )
        else:
            sampler = None
        loader = DataLoader(
            self.dataset,  # num_workers=self.queue_workers, #in multigpu, >0 not wrkng
            batch_size=1, collate_fn=self._get_first_item, pin_memory=True,
            shuffle=False, sampler=sampler
        )
        self._num_sampled_images = 0
        return iter(loader)

    def _get_next_image(self):
        try:
            image = next(self._images_iterable)
        except Exception as e:
            print(f"[WARN]:Exception while getting image for patching: {e}")
            self.initialize_images_iterable()
            image = next(self._images_iterable)
        return image

    def extract_patches(self, image_pair, samples_per_image, patch_size):
        image_patches = []
        img_height = image_pair['image'].shape[-2]
        img_width = image_pair['image'].shape[-1]
        # Check if image size is smaller than patch size
        if img_height < patch_size[0] or img_width < patch_size[1]:
            # Resize the image to match the patch size
            image_pair["image"] = self.resize_transform(image_pair["image"])
            image_pair["mask"] = self.resize_transform(image_pair["mask"])
            img_height = image_pair["image"].shape[-2]
            img_width = image_pair["image"].shape[-1]
        for _ in range(samples_per_image):
            left = torch.randint(
                low=0, high=img_width-patch_size[0], size=[1,]
            ).item()
            top = torch.randint(
                low=0, high=img_height-patch_size[1], size=[1,]
            ).item()
            cropped_image = crop(
                img=image_pair["image"], top=top, left=left,
                height=patch_size[1], width=patch_size[0]
            )
            cropped_labels = crop(
                img=image_pair["mask"], top=top, left=left,
                height=patch_size[1], width=patch_size[0]
            )
            image_patches.append({"image": cropped_image,
                                  "mask": cropped_labels})
        return image_patches

    def _fill(self):
        while True:
            image_pair = self._get_next_image()
            samples_per_image = self.samples_per_image
            patch_size = self.patch_size
            patches = self.extract_patches(
                image_pair, samples_per_image, patch_size
            )
            self.patch_list.extend(patches)
            self._num_sampled_images += 1
            islistfull = len(self.patch_list) >= self.max_length
            if islistfull:
                break

In [12]:
def get_transforms():
    # Transforms
    image_transforms = tvt.Compose([tvt.ToTensor(),])
    mask_transforms = tvt.Compose([tvt.PILToTensor(),
                                  RemapBackground(),
                                  RemapLabels(),])
    return image_transforms, mask_transforms

In [13]:
class CustomDataset(Dataset):
    def __init__(self, dataset, transforms=None):
        self.dataset = dataset
        self.transforms = transforms

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]["image"]
        semantic_mask = self.dataset[idx]["semantic_mask_label"]
        if self.transforms:
            image_transforms, mask_transforms = self.transforms
            transformed_image = image_transforms(image)
            transformed_mask = mask_transforms(semantic_mask)
        else:
            image_transforms, mask_transforms = get_transforms()
            transformed_image = image_transforms(image)
            transformed_mask = mask_transforms(semantic_mask)
        return {
            "image": transformed_image, "mask": transformed_mask
        }

In [14]:
def _collate_fn(batch):
    images = torch.stack([sample["image"] for sample in batch])
    segmentation_maps = torch.cat([sample["mask"] for sample in batch])
    batch = processor(images, segmentation_maps=segmentation_maps, task_inputs=["semantic"] * len(images), return_tensors="pt")
    batch["original_images"] = images
    batch["original_segmentation_maps"] = segmentation_maps
    return batch

def _prepare_dataloader(dataset: torch.utils.data.Dataset):
    sampler = None
    is_train=True
    dataloader = DataLoader(dataset, batch_size=2, pin_memory=True, shuffle=(is_train and sampler is None), sampler=sampler, collate_fn=_collate_fn)
    return dataloader

####Token Embedding:

The text tensor represents the input tokens with shape `(batch_size*234, 77)`, where 234 is the number of text sequences and 77 is the maximum sequence length.
The token_embedding layer maps each token to a continuous vector representation. After passing through this layer, hidden_state will have a shape of `(batch_size*234, 77, width)`.

####Positional Embedding Addition:

The positional_embedding parameter provides positional information to the input tokens to help the model understand their position in the sequence.
This positional embedding is added to the token embeddings element-wise, maintaining the shape of hidden_state as (batch_size*234, 77, width).
####Permutation:

The dimensions of the hidden_state tensor are permuted to bring the batch dimension to the second position. After permutation, the shape becomes (77, batch_size*234, width).
####Transformer Layer:

The hidden_state tensor is passed through the transformer model (self.transformer). The transformer processes the input tokens, capturing their contextual dependencies.
The output of the transformer layer retains the shape (77, batch_size*234, width).
####Permutation (Back to Original Shape):

After the transformer layer, the dimensions are permuted back to the original shape, with the batch dimension in the first position. Now, the shape is (batch_size*234, 77, width).
####Layer Normalization:

The ln_final layer applies layer normalization along the last dimension of the hidden_state tensor. This layer ensures that the mean activation of each neuron is close to 0 and the standard deviation is close to 1.
After layer normalization, the shape remains (batch_size*234, 77, width).
####Extract Token Representations:

Finally, the token representation for each text sequence is extracted. This is done by selecting the token with the highest value along the last dimension (argmax operation) since this dimension corresponds to the token embeddings.
The resulting hidden_state tensor has a shape of (batch_size*234, width), where each row represents the token representation for a specific text sequence.

####Note on permutation on step 3
The permutation step in the forward function of the OneFormerTextEncoder module is necessary to ensure that the input tensor has the correct shape before being passed through the transformer layer. Let's break down why this permutation is needed:

- Initial Embedding: Initially, the input tensor hidden_state has the shape (batch_size*234, 77, width). This shape corresponds to (sequence_length, batch_size, embedding_dim), where:

- sequence_length represents the length of each text sequence (77 tokens in this case).
batch_size*234 represents the total number of text sequences in the batch, obtained by concatenating the text sequences along the batch dimension.
Permutation: The permutation step hidden_state.permute(1, 0, 2) changes the order of dimensions in the tensor. Specifically, it rearranges the dimensions such that:

The dimension corresponding to the sequence length (77) becomes the first dimension.
The dimension corresponding to the batch size multiplied by the number of text sequences (batch_size*234) becomes the second dimension.
The embedding dimension (width) remains unchanged as the third dimension.
After permutation, the shape of the tensor becomes (77, batch_size*234, width), which is the expected shape for input to the transformer layer.

Transformer Layer: The permuted tensor is then passed through the transformer layer, which operates on this input shape (sequence_length, batch_size*234, width).
https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html
In summary, the permutation step ensures that the input tensor is properly shaped to match the requirements of the transformer layer, enabling correct processing of the text sequences.

``` override oneformer processor _preprocess_text to get input_masks and attention_mask```

```
from transformers import CLIPProcessor

class CustomOneFormerProcessor(OneFormerProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def _preprocess_text(self, text_list=None, max_length=77, text_inputs=False):
        # Override the method to change the preprocessing logic
        if text_list is None:
            raise ValueError("tokens cannot be None.")

        tokens = self.tokenizer(text_list, padding="max_length", max_length=max_length, truncation=True)

        attention_masks, input_ids = tokens["attention_mask"], tokens["input_ids"]
        
        if text_inputs:
           return input_ids, attention_masks

        token_inputs = []
        for attn_mask, input_id in zip(attention_masks, input_ids):
            token = torch.tensor(attn_mask) * torch.tensor(input_id)
            token_inputs.append(token.unsqueeze(0))

        token_inputs = torch.cat(token_inputs, dim=0)
        
        return token_inputs

    def __call__(self, images=None, task_inputs=None, segmentation_maps=None, **kwargs):

        if task_inputs is None:
            raise ValueError("You have to specify the task_input. Found None.")
        elif images is None:
            raise ValueError("You have to specify the image. Found None.")

        if not all(task in ["semantic", "instance", "panoptic"] for task in task_inputs):
            raise ValueError("task_inputs must be semantic, instance, or panoptic.")

        encoded_inputs = self.image_processor(images, task_inputs, segmentation_maps, **kwargs)

        if isinstance(task_inputs, str):
            task_inputs = [task_inputs]

        if isinstance(task_inputs, List) and all(isinstance(task_input, str) for task_input in task_inputs):
            task_token_inputs = []
            for task in task_inputs:
                task_input = f"the task is {task}"
                task_token_inputs.append(task_input)
            encoded_inputs["task_inputs"] = self._preprocess_text(task_token_inputs, max_length=self.task_seq_length)
        else:
            raise TypeError("Task Inputs should be a string or a list of strings.")

        if hasattr(encoded_inputs, "text_inputs"):
            texts_list = encoded_inputs.text_inputs

            text_inputs = []
            for texts in texts_list:
                text_input_list = self._preprocess_text(texts, max_length=self.max_seq_length, text_inputs=True)
                # text_inputs.append(text_input_list.unsqueeze(0))
                text_inputs.append(text_input_list)

            encoded_inputs["text_inputs"] = torch.cat(text_inputs, dim=0)

        return encoded_inputs

    def encode_inputs(self, images=None, task_inputs=None, segmentation_maps=None, **kwargs):
        """
        This method forwards all its arguments to [`OneFormerImageProcessor.encode_inputs`] and then tokenizes the
        task_inputs. Please refer to the docstring of this method for more information.
        """

        if task_inputs is None:
            raise ValueError("You have to specify the task_input. Found None.")
        elif images is None:
            raise ValueError("You have to specify the image. Found None.")

        if not all(task in ["semantic", "instance", "panoptic"] for task in task_inputs):
            raise ValueError("task_inputs must be semantic, instance, or panoptic.")

        encoded_inputs = self.image_processor.encode_inputs(images, task_inputs, segmentation_maps, **kwargs)

        if isinstance(task_inputs, str):
            task_inputs = [task_inputs]

        if isinstance(task_inputs, List) and all(isinstance(task_input, str) for task_input in task_inputs):
            task_token_inputs = []
            for task in task_inputs:
                task_input = f"the task is {task}"
                task_token_inputs.append(task_input)
            encoded_inputs["task_inputs"] = self._preprocess_text(task_token_inputs, max_length=self.task_seq_length)
        else:
            raise TypeError("Task Inputs should be a string or a list of strings.")

        if hasattr(encoded_inputs, "text_inputs"):
            texts_list = encoded_inputs.text_inputs

            text_inputs = []
            for texts in texts_list:
                text_input_list = self._preprocess_text(texts, max_length=self.max_seq_length, text_inputs=True)
                # text_inputs.append(text_input_list.unsqueeze(0))
                text_inputs.append(text_input_list)

            encoded_inputs["text_inputs"] = torch.cat(text_inputs, dim=0)

        return encoded_inputs
```

- make `oneformer.text_mapper.text_encoder_width = 512` for oneformer and then project it to 256 dimension. we can perform feature distillation in this case
- use clip to create text embedding for each text of image
- use the transformer using in oneformer, but reduce the layers of the text mapper transformer using `text_encoder_num_layers` may be to 2 . keep in ind of attention heads creeating from embedding dimension
- attention heads = width // 64 = 512 // 64 = 8 , 256 // 64 = 4

In [15]:
# import torch
# from torch import nn

# batch, sentence_length, embedding_dim = 20, 5, 10
# embedding = torch.randn(batch, sentence_length, embedding_dim)
# layer_norm = nn.LayerNorm(embedding_dim)
# hidden_state = layer_norm(embedding) #20, 5, 10
# hidden_state.shape, hidden_state

In [16]:
from torch import nn, Tensor
from transformers import OneFormerConfig
from transformers.models.oneformer.modeling_oneformer import OneFormerTextMapper, OneFormerTextEncoder, OneFormerMLPPredictionHead


# Define your custom subclass of OneFormerTextMapper to override encode_text to return distillation logits
# FIXME: Can we improve this by moving embedding layer to OneFormerModel and concat there?
class CustomOneFormerTextMapper(OneFormerTextMapper):
    def __init__(self, config):
        super().__init__(config)
        if config.text_encoder_n_ctx > 0:
            self.prompt_ctx = nn.Embedding(
                    config.text_encoder_n_ctx,
                    # config.text_encoder_width,
                    config.hidden_dim, #config.text_encoder_width change it to hidden_dim as we are chaging text_encoder_width = 512 , withoutit we get concatentation failure in forward bcz of shape mismatch of projected encoded text
                )

    def forward(self, inputs: Tensor) -> Tensor:
        text_queries, text_queries_distillation_logits = self.encode_text(inputs)
        return text_queries, text_queries_distillation_logits

    def encode_text(self, text):
        print("custom text mapper encoder output")
        if text.ndim is None:
            raise ValueError("text must not be NoneType")
        if text.ndim not in [2, 3]:
            raise ValueError("Number of dimensions in text must be 2 or 3")
        squeeze_dim = False
        num_text = 1
        if text.ndim == 3:
            num_text = text.shape[1]
            batch_size, num_text, hidden_dim = text.shape
            print("text shape in encode_text textmapper- (bs, num_text, hidden_dim)")
            print(text.shape)
            text = text.reshape(batch_size * num_text, hidden_dim)
            squeeze_dim = True

        encoded_text = self.text_encoder(text)  # bsx234 , width =  bsx234, 512 for ours
        print("encoded_text")
        print(encoded_text.shape)
        _, hidden_dim = encoded_text.shape
        text_queries_before_ctx = encoded_text.reshape(batch_size, num_text, hidden_dim) #bs, 234, 512
        print("text_queries_before_ctx")
        print(text_queries_before_ctx.shape)
        #projection
        text_queries = self.text_projector(encoded_text)  # bsx234, 256
        print("text_queries after projection")
        print(text_queries.shape)
        if squeeze_dim:
            _, hidden_dim = text_queries.shape
            text_queries = text_queries.reshape(batch_size, num_text, hidden_dim)
            if self.prompt_ctx is not None:
                text_queries_ctx = self.prompt_ctx.weight.unsqueeze(0).repeat(text_queries.shape[0], 1, 1)
                text_queries = torch.cat([text_queries, text_queries_ctx], dim=1)
                # RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 256 but got size 512 for tensor number 1 in the list.
        return text_queries, text_queries_before_ctx

In [17]:
from transformers.models.oneformer.modeling_oneformer import OneFormerModelOutput
from typing import Optional
from dataclasses import dataclass
from transformers.utils import ModelOutput

@dataclass
class CustomOneFormerModelOutput(ModelOutput):
    print("inside custm one former model output")
    encoder_hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    pixel_decoder_hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    transformer_decoder_hidden_states: Optional[torch.FloatTensor] = None
    transformer_decoder_object_queries: torch.FloatTensor = None
    transformer_decoder_contrastive_queries: Optional[torch.FloatTensor] = None
    transformer_decoder_mask_predictions: torch.FloatTensor = None
    transformer_decoder_class_predictions: torch.FloatTensor = None
    transformer_decoder_auxiliary_predictions: Optional[Tuple[Dict[str, torch.FloatTensor]]] = None
    text_queries: Optional[torch.FloatTensor] = None
    text_queries_distillation_logits: Optional[torch.FloatTensor] = None  # new
    task_token: torch.FloatTensor = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

inside custm one former model output


In [18]:
from torch import nn, Tensor
from transformers import OneFormerConfig
from transformers.models.oneformer.modeling_oneformer import OneFormerModel, OneFormerModelOutput, OneFormerLoss


class CustomOneFormerModel(OneFormerModel):
    def __init__(self, config):
        super().__init__(config)
        if self.is_training:
            self.text_mapper = CustomOneFormerTextMapper(config)
        else:
            self.text_mapper = None

    def forward(self, pixel_values, task_inputs, text_inputs=None, pixel_mask=None, output_hidden_states=None, output_attentions=None, return_dict=None,) -> CustomOneFormerModelOutput:
        print("inside my custom model")
        if pixel_values is None:
            raise ValueError("You have to specify pixel_values")

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states)
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        batch_size, _, height, width = pixel_values.shape

        if pixel_mask is None:
            pixel_mask = torch.ones((batch_size, height, width), device=pixel_values.device)

        pixel_level_module_output = self.pixel_level_module(pixel_values, output_hidden_states)

        multi_scale_features = pixel_level_module_output.decoder_features
        mask_features = pixel_level_module_output.decoder_last_feature

        task_token = self.task_encoder(task_inputs.to(self.dtype))

        if self.is_training:
            text_queries, text_queries_distillation_logits = self.text_mapper(text_inputs)  # new
        else:
            text_queries = None
            text_queries_distillation_logits = None

        transformer_module_output = self.transformer_module(
            multi_scale_features=multi_scale_features,
            mask_features=mask_features,
            task_token=task_token,
            output_attentions=output_attentions,
        )

        queries = transformer_module_output.object_queries

        encoder_hidden_states = None
        pixel_decoder_hidden_states = None
        transformer_decoder_hidden_states = None

        if output_hidden_states:
            encoder_hidden_states = pixel_level_module_output.encoder_features
            pixel_decoder_hidden_states = (pixel_level_module_output.decoder_last_feature,)
            for f in pixel_level_module_output.decoder_features:
                pixel_decoder_hidden_states += (f,)
            transformer_decoder_hidden_states = transformer_module_output.auxiliary_predictions

        output = CustomOneFormerModelOutput(
            encoder_hidden_states=encoder_hidden_states,
            pixel_decoder_hidden_states=pixel_decoder_hidden_states,
            transformer_decoder_hidden_states=transformer_decoder_hidden_states,
            transformer_decoder_object_queries=queries,
            transformer_decoder_contrastive_queries=transformer_module_output.contrastive_logits,
            transformer_decoder_mask_predictions=transformer_module_output.prediction_masks,
            transformer_decoder_class_predictions=transformer_module_output.prediction_class,
            transformer_decoder_auxiliary_predictions=transformer_module_output.auxiliary_predictions,
            text_queries=text_queries,
            text_queries_distillation_logits=text_queries_distillation_logits,  # new
            task_token=task_token,
            attentions=transformer_module_output.attentions,
        )

        if not return_dict:
            output = tuple(v for v in output.values())

        return output

In [19]:
@dataclass
class CustomOneFormerForUniversalSegmentationOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    class_queries_logits: torch.FloatTensor = None
    masks_queries_logits: torch.FloatTensor = None
    auxiliary_predictions: List[Dict[str, torch.FloatTensor]] = None
    encoder_hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    pixel_decoder_hidden_states: Optional[List[torch.FloatTensor]] = None
    transformer_decoder_hidden_states: Optional[torch.FloatTensor] = None
    transformer_decoder_object_queries: torch.FloatTensor = None
    transformer_decoder_contrastive_queries: Optional[torch.FloatTensor] = None
    transformer_decoder_mask_predictions: torch.FloatTensor = None
    transformer_decoder_class_predictions: torch.FloatTensor = None
    transformer_decoder_auxiliary_predictions: Optional[List[Dict[str, torch.FloatTensor]]] = None
    text_queries: Optional[torch.FloatTensor] = None
    text_queries_distillation_logits: Optional[torch.FloatTensor] = None  # new
    task_token: torch.FloatTensor = None
    attentions: Optional[Tuple[Tuple[torch.FloatTensor]]] = None

In [30]:
from torch import nn
def sigmoid_cross_entropy_loss(inputs: torch.Tensor, labels: torch.Tensor, num_masks: int) -> torch.Tensor:
    #new
    # resized_inputs = nn.functional.interpolate(inputs, size=(labels.size(-2), labels.size(-1)), mode="nearest") #, align_corners=False ##ValueError: Target size (torch.Size([13, 1, 512, 512])) must be the same as input size (torch.Size([13, 1, 128, 128])) when predmasks and target masks are sent
    # print(f"resized_inputs: {resized_inputs.shape}") #torch.Size([17, 1, 512, 512])

    print(f"sigmoid_cross_entropy_loss_fn - inputs: {inputs.shape}, labels: {labels.shape}, num_masks: {num_masks}")
    #inputs: torch.Size([17, 12544]), labels: torch.Size([17, 12544]), num_masks: tensor([17.], device='cuda:0')
    # for resized_inputs - torch.Size([17, 1, 128, 128]) #torch.Size([17, 1, 512, 512]) #tensor([17.]

    criterion = nn.BCEWithLogitsLoss(reduction="none")
    # apply ohem here
    cross_entropy_loss = criterion(inputs, labels) # resized_inputs
    print(f"cross_entropy_loss: {cross_entropy_loss.shape}, {cross_entropy_loss}")
    # torch.Size([17, 12544])
    # for resized_inputs - torch.Size([17, 1, 512, 512])

    loss = cross_entropy_loss.mean(1).sum() / num_masks
    print(f"mean_cross_entropy_loss: {loss}") # mean_cross_entropy_loss: tensor([0.4132])
    return loss

def dice_loss(inputs: Tensor, labels: Tensor, num_masks: int) -> Tensor:
    print(f"diceloss: inputs - {inputs.shape}, labels - {labels.shape}, num_masks: {num_masks.shape}")
    # torch.Size([17, 12544]), torch.Size([17, 12544]), tensor([17.], num_masks: torch.Size([1])??? bug?

    probs = inputs.sigmoid().flatten(1)
    print(f"probs: {probs.shape}") # torch.Size([17, 12544])

    numerator = 2 * (probs * labels).sum(-1)
    denominator = probs.sum(-1) + labels.sum(-1)
    print(f"numerator: {numerator}, denominator:{denominator}") # torch.Size([17]), torch.Size([17])
    #example numerator: tensor([1.3901e+04, 4.6304e+02, 1.4267e+03, 1.3075e+04, 7.6254e+01, 3.5534e+03, 5.6537e+03, 1.2245e+02, 1.6539e+03, 4.5592e+02, 4.6478e-01, 1.6648e+03, 2.6228e+02, 7.2001e+03]),
    #example denominator:tensor([17006.8262,   669.5397,  1667.3591, 13333.7393,   247.1372,  6696.2036, 11886.2363,   280.5628,  1848.1116,  3018.4907,    87.1838,  4434.2354, 3855.7500, 11448.5762])

    loss = 1 - (numerator + 1) / (denominator + 1)
    loss = loss.sum() / num_masks
    # apply ohem here
    print(f"dice_loss: {loss.shape}, {loss}") # dice_loss: torch.Size([17]) #tensor([0.2231, 0.0690, 0.9012, 0.5166, 0.7142, 0.4936, 0.1366, 0.4007, 0.5544, 0.0995, 0.3701, 0.7276, 0.0405, 0.4789, 0.2160, 0.6159, 0.1205]
    # dice_loss: torch.Size([1]), tensor([0.4830])??
    print(f"mean_dice_loss:  {loss}") # mean_dice_loss: tensor([0.3928])
    return loss

In [29]:
def sample_point(input_features: torch.Tensor, point_coordinates: torch.Tensor, add_dim=False, **kwargs) -> torch.Tensor:
    print(f"sample point: input_features: {input_features.shape}, point_coordinates: {point_coordinates.shape}")
    # input_features: torch.Size([17, 1, 128, 128]), point_coordinates: torch.Size([17, 37632, 2]) #for pred_mask from sample_points_using_uncertainty before calling sample_point, oversampling
    # input_features: torch.Size([17, 1, 512, 512]), point_coordinates: torch.Size([17, 12544, 2]) #for targetlabel from sample_point
    # input_features: torch.Size([17, 1, 128, 128]), point_coordinates: torch.Size([17, 12544, 2]) #for pred_mask from sample_point
    if point_coordinates.dim() == 3:
        add_dim = True
        point_coordinates = point_coordinates.unsqueeze(2)
        print(f"inside point_coordinates.dim() == 3, point_features: {point_coordinates.shape}")
        #point_features: torch.Size([17, 37632, 1, 2]) #for pred_mask from sample_points_using_uncertainty before calling sample_point, oversample
        #point_features: torch.Size([17, 12544, 1, 2]) #for targetlabel from sample_point
        #point_features: torch.Size([17, 12544, 1, 2]) #for pred_mask from sample_point
    point_features = torch.nn.functional.grid_sample(input_features, 2.0 * point_coordinates - 1.0, **kwargs) # point_coordinates: This tensor contains the coordinates of the points where the sampling should be performed. It's assumed to be in the range of [-1, 1] for both dimensions, where (-1, -1) represents the top-left corner of the input feature map, and (1, 1) represents the bottom-right corner. The expression 2.0 * point_coordinates - 1.0 is used to normalize the coordinates to the range [-1, 1].
    print(f"point_features: {point_features.shape}")
    #torch.Size([17, 1, 37632, 1]) ##for pred_mask from sample_points_using_uncertainty before calling sample_point, oversample
    #torch.Size([17, 1, 12544, 1]) ##for targetlabel from sample_point
    #torch.Size([17, 1, 12544, 1]) ##for pred_mask from sample_point
    if add_dim:
        point_features = point_features.squeeze(3)
        print(f"inside add_dim, point_features: {point_features.shape}")
        #torch.Size([17, 1, 37632]) ###for pred_mask from sample_points_using_uncertainty before calling sample_point, oversample
        #torch.Size([17, 1, 12544]) ###for targetlabel from sample_point
        #torch.Size([17, 1, 12544]) ###for pred_mask from sample_point
    return point_features

In [ ]:
class OhemCELoss(nn.Module):

    def __init__(self, score_thresh, n_min=None, ignore_index=255):
        super(OhemCELoss, self).__init__()
        self.score_thresh = score_thresh
        self.ignore_lb = ignore_index
        self.n_min = n_min
        self.criteria = nn.CrossEntropyLoss(ignore_index=ignore_index, reduction='mean')

    def forward(self, logits, labels):
        n_min = labels.numel() // 16 if self.n_min is None else self.n_min
        labels = ohem_cpp.score_ohem_label(logits.float(), labels,
                self.ignore_lb, self.score_thresh, n_min).detach()
        loss = self.criteria(logits, labels)
        return loss

In [ ]:
from torch import nn
loss_manual_no_reduction = nn.CrossEntropyLoss(weight=ce_class_weights, reduction="none")
loss_ce_without_reduction = loss_manual_no_reduction(pred_logits_transposed, target_classes) # (2, 17, 250), (2, 250)
print(f"loss_ce_without_reduction: {loss_ce_without_reduction.shape}, {loss_ce_without_reduction}") # torch.Size([2, 250])
loss_ce_manual_mean = loss_ce_without_reduction.sum() / self.empty_weight[target_classes].sum()

class OhemCrossEntropy2d(nn.Module):
    def __init__(self, thresh=0.7, min_kept=0, ignore_index=255, is_binary=False, **kwargs):
        super().__init__()
        self.ignore_label = ignore_index
        self.is_binary = is_binary
        self.thresh = float(thresh)
        self.min_kept = int(min_kept)
        self.criterion = nn.CrossEntropyLoss(**kwargs) #https://discuss.pytorch.org/t/softmax-cross-entropy-loss/125383

    def forward(self, logits, labels, **_):
        """
            Args:
                predict:(n, c, h, w), pred_logits_transposed: (n, num_labels, num_queries)
                labels:(n, h, w), target_labels: (n, num_queries)
        """

        if self.is_binary:
            predict = torch.sigmoid(logits)
        else:
            predict = F.softmax(logits, dim=1)

        n, c, h, w = predict.size()
        input_label = labels.detach().cpu().numpy().ravel().astype(np.int32)
        x = np.rollaxis(predict.detach().cpu().numpy(), 1).reshape((c, -1))
        input_prob = np.exp(x - x.max(axis=0).reshape((1, -1)))
        input_prob /= input_prob.sum(axis=0).reshape((1, -1))

        valid_flag = input_label != self.ignore_label
        valid_inds = np.where(valid_flag)[0]
        label = input_label[valid_flag]
        num_valid = valid_flag.sum()
        if self.min_kept >= num_valid:
            print('Labels: {}'.format(num_valid))
        elif num_valid > 0:
            prob = input_prob[:, valid_flag]
            pred = prob[label-1, np.arange(len(label), dtype=np.int32)]
            threshold = self.thresh
            if self.min_kept > 0:
                index = pred.argsort()
                threshold_index = index[min(len(index), self.min_kept) - 1]
                if pred[threshold_index] > self.thresh:
                    threshold = pred[threshold_index]
            kept_flag = pred <= threshold
            valid_inds = valid_inds[kept_flag]
            #print('hard ratio: {} = {} / {} '.format(round(len(valid_inds)/num_valid, 4), len(valid_inds), num_valid))

        label = input_label[valid_inds].copy()
        input_label.fill(self.ignore_label)
        input_label[valid_inds] = label
        #print(np.sum(input_label != self.ignore_label))
        labels = torch.from_numpy(input_label.reshape(labels.size())).type_as(predict).to(labels.device)

        predict = predict.squeeze()          # in case we're dealing with B/W images instead of RGB
        return self.criterion(predict, labels)

In [ ]:
from torch import nn

# loss_manual_no_reduction = nn.CrossEntropyLoss(weight=ce_class_weights, reduction="none")
# loss_ce_without_reduction = loss_manual_no_reduction(pred_logits_transposed, target_classes) # (2, 17, 250), (2, 250)
# print(f"loss_ce_without_reduction: {loss_ce_without_reduction.shape}, {loss_ce_without_reduction}") # torch.Size([2, 250])
# loss_ce_manual_mean = loss_ce_without_reduction.sum() / self.empty_weight[target_classes].sum()

class OhemCELoss(nn.Module):
    """
    OhemCELoss - Online Hard Example Mining Cross Entropy Loss
    """

    def __init__(self,
                 weights: torch.Tensor,
                 threshold: float=0.4,
                 mining_percent: float=0.8,
                 ignore_index: int=255,
                 num_masks_exclude_ignored: bool = True):
        """
        Args:
            threshold: Sample below probability threshold, is considered hard.
            num_masks_exclude_ignored: How to calculate total masks from which extract mining percent of the samples
            ignore_index: label index to be ignored in loss calculation.
            criteria: loss to mine the examples from.

          Example: for num_masks=100, ignore_masks=30(from ignore_index label), mining_percent=0.1:
                        num_masks_exclude_ignored=False => num_mining = 100 * 0.1 = 10
                        num_masks_exclude_ignored=True  => num_mining = (100 - 30) * 0.1 = 7
        """
        super().__init__()

        if mining_percent < 0 or mining_percent > 1:
            raise ValueError(f"mining percent should be between (0, 1] but given {mining_percent}")

        self.thresh = -torch.log(torch.tensor(threshold, dtype=torch.float))
        self.mining_percent = mining_percent
        self.ignore_index = 255 if ignore_index is None or ignore_index < 0 else ignore_index
        self.num_masks_exclude_ignored = num_masks_exclude_ignored
        self.class_weights = weights
        self.criteria = nn.CrossEntropyLoss(weight=self.class_weights, ignore_index=ignore_index, reduction="none") #https://discuss.pytorch.org/t/softmax-cross-entropy-loss/125383
        if self.criteria.reduction != "none":
            raise ValueError(f"Criteria reduction should be none but given {self.criteria.reduction}")


    def forward(self, logits, labels):
        # pred_logits_transposed => logits : (n, num_labels_in_batch, num_queries)
        # target_labels=> labels : (n, num_queries)

        loss = self.criteria(logits, labels).view(-1)


       # Flatten the labels tensor
       flattened_labels = labels.view(-1)

        # Excluded Masks of ignore_index class label
        if self.num_masks_exclude_ignored:
            ignore_masks = labels.view(-1) != self.ignore_index
            flattened_labels = labels[ignore_masks]
            loss = loss[ignore_masks] # remove ignore label elements
            num_masks = loss.numel() # total num masks in a batch -> num_masks = batch_size * num_masks_in_batch * num_queries - ignore_masks
        else:
            num_masks = labels.numel()

        if num_masks == 0: # if all masks are ignore labels, return empty loss tensor
            return torch.tensor([0.0]).requires_grad_(True)

        #Hard Mining
        num_mining = int(self.mining_percent * num_masks)
        num_mining = min(num_mining, num_masks - 1) # in case mining_percent=1, prevent out of bound exception

        self.thresh = self.thresh.to(logits.device)
        # orting
        loss, sorted_indices = torch.sort(loss, descending=True)
        sorted_labels = flattened_labels[sorted_indices]

        if loss[num_mining] > self.thresh: #check if value at num_mining index is > thresh
            mask = loss > self.thresh
            loss = loss[mask] #get the loss where the mask is true #original shape
            selected_labels = sorted_labels[mask]
        else:
            loss = loss[:num_mining] # num_mining #If there are fewer loss values above the threshold than num_mining, it selects the top num_mining loss values.
            selected_labels = sorted_labels[:num_mining]

        if len(loss) == 0:  # Check if no loss values are selected i.e no loss values are greater than threshold
           return torch.tensor([0.0]).requires_grad_(True)

        # Calculate the class weights for the selected masks
        # selected_labels = labels.view(-1)[loss.view(-1) > self.thresh]

        #calculate weighted meanloss
        loss_mean = loss.sum() / self.class_weights[selected_labels].sum() #we need to get labels after mining
        return loss_mean


In [37]:
class CustomOneFormerLoss(OneFormerLoss):
    def __init__(self, *args, **kwargs, class_specific_weights):
        super().__init__(*args, **kwargs)
        print(f"Empty Weight: {self.empty_weight}") #existing weights logic,ones to all classes

        if class_specific_weights is not None: #new for class weights #adding also weight for bg class
            class_specific_weights.append(self.eos_coef)
        self.register_buffer("class_specific_weights", class_specific_weights)


    def calculate_uncertainty(self, logits: torch.Tensor) -> torch.Tensor:
        """
        In Mask2Former paper, uncertainty is estimated as L1 distance between 0.0 and the logit prediction in 'logits' for the foreground class in `classes`.
        Args:
            logits (`torch.Tensor`): A tensor of shape (R, 1, ...) for class-specific or class-agnostic, where R is the total number of predicted masks in all images and C is:
            the number of foreground classes. The values are logits.
        Returns:
            scores (`torch.Tensor`): A tensor of shape (R, 1, ...) that contains uncertainty scores with the most uncertain locations having the highest uncertainty score.
        """
        uncertainty_scores = -(torch.abs(logits))
        return uncertainty_scores

    def sample_points_using_uncertainty(self, logits: torch.Tensor, uncertainty_function, num_points: int, oversample_ratio: int, importance_sample_ratio: float,) -> torch.Tensor:
        """
        This function is meant for sampling points in [0, 1] * [0, 1] coordinate space based on their uncertainty. The
        uncertainty is calculated for each point using the passed `uncertainty function` that takes points logit
        prediction as input.
        Args:
            logits (`float`): Logit predictions for P points.
            uncertainty_function: A function that takes logit predictions for P points and returns their uncertainties.
            num_points (`int`): The number of points P to sample.
            oversample_ratio (`int`): Oversampling parameter.
            importance_sample_ratio (`float`): Ratio of points that are sampled via importance sampling.
        Returns:
            point_coordinates (`torch.Tensor`): Coordinates for P sampled points.
        """
        print("sample_points_using_uncertainty")
        num_boxes = logits.shape[0] # torch.Size([17, 1, 128, 128])
        print(f"num_boxes - {num_boxes}") #num_boxes - 17

        num_points_sampled = int(num_points * oversample_ratio) # 12544 * 3.0
        print(f"num_points_sampled- {num_points_sampled}") # 37632

        # Get random point coordinates
        point_coordinates = torch.rand(num_boxes, num_points_sampled, 2, device=logits.device)
        print(f"point_coordinates - {point_coordinates.shape}") # point_coordinates - torch.Size([17, 37632, 2])
        # Get sampled prediction value for the point coordinates
        point_logits = sample_point(logits, point_coordinates, align_corners=False)
        print(f"point_logits - {point_logits.shape}") # torch.Size([17, 1, 37632])
        # Calculate the uncertainties based on the sampled prediction values of the points
        point_uncertainties = uncertainty_function(point_logits) #
        print(f"point_uncertainties - {point_uncertainties.shape}") # torch.Size([17, 1, 37632])
        num_uncertain_points = int(importance_sample_ratio * num_points) #0.75 * 12544 = 9408
        print(f"num_uncertain_points: {num_uncertain_points}") #num_uncertain_points: 9408
        num_random_points = num_points - num_uncertain_points #12544 - 9408
        print(f"num_random_points: {num_random_points}") #num_random_points: 3136

        idx = torch.topk(point_uncertainties[:, 0, :], k=num_uncertain_points, dim=1)[1] #
        print(f"idx: {idx.shape}") # torch.Size([17, 9408])
        #idx: tensor([[23785, 27780, 31340,  ..., 28887,  8046, 30607],
        # [18878, 13895, 31258,  ..., 28003,  6732,  2762],
        # [ 7469, 19368, 21082,  ..., 19072, 14708, 11367],
        # ...,
        # [18381, 33648, 35147,  ..., 32872,   717, 21641],
        # [  321, 24568, 10217,  ..., 11078, 27017, 13298],
        # [32526, 12735, 34185,  ..., 13917, 17858, 12344]], device='cuda:0')
        shift = num_points_sampled * torch.arange(num_boxes, dtype=torch.long, device=logits.device)
        print(f"shift: {shift.shape}") #shift: torch.Size([17])
        idx += shift[:, None]
        point_coordinates = point_coordinates.view(-1, 2)[idx.view(-1), :].view(num_boxes, num_uncertain_points, 2)
        print(f"point_coordinates: {point_coordinates.shape}") #torch.Size([17, 9408, 2])
        if num_random_points > 0:
            point_coordinates = torch.cat(
                [point_coordinates, torch.rand(num_boxes, num_random_points, 2, device=logits.device)],
                dim=1,
            )
            print(f"inside num_random_points>0, point_coordinates: {point_coordinates.shape}") #torch.Size([17, 12544, 2])
        return point_coordinates

    def _get_predictions_permutation_indices(self, indices):
        # permute predictions following indices
        batch_indices = torch.cat([torch.full_like(src, i) for i, (src, _) in enumerate(indices)])
        predictions_indices = torch.cat([src for (src, _) in indices])
        return batch_indices, predictions_indices

    def _get_targets_permutation_indices(self, indices):
        # permute labels following indices
        batch_indices = torch.cat([torch.full_like(tgt, i) for i, (_, tgt) in enumerate(indices)])
        target_indices = torch.cat([tgt for (_, tgt) in indices])
        return batch_indices, target_indices

    def _max_by_axis(self, the_list: List[List[int]]) -> List[int]:
        maxes = the_list[0]
        for sublist in the_list[1:]:
            for index, item in enumerate(sublist):
                maxes[index] = max(maxes[index], item)
        return maxes

    def _pad_images_to_max_in_batch(self, tensors: List[Tensor]) -> Tuple[Tensor, Tensor]:
        # get the maximum size in the batch
        max_size = self._max_by_axis([list(tensor.shape) for tensor in tensors])
        batch_size = len(tensors)
        # compute finel size
        batch_shape = [batch_size] + max_size
        b, _, h, w = batch_shape
        # get metadata
        dtype = tensors[0].dtype
        device = tensors[0].device
        padded_tensors = torch.zeros(batch_shape, dtype=dtype, device=device)
        padding_masks = torch.ones((b, h, w), dtype=torch.bool, device=device)
        # pad the tensors to the size of the biggest one
        for tensor, padded_tensor, padding_mask in zip(tensors, padded_tensors, padding_masks):
            padded_tensor[: tensor.shape[0], : tensor.shape[1], : tensor.shape[2]].copy_(tensor)
            padding_mask[: tensor.shape[1], : tensor.shape[2]] = False

        return padded_tensors, padding_masks

    def loss_labels(self, class_queries_logits: Tensor, class_labels: List[Tensor], indices: Tuple[np.array]) -> Dict[str, Tensor]:
        pred_logits = class_queries_logits
        print(f"loss_labels - class_queries_logits or pred_logits: {pred_logits.shape}") # torch.Size([2, 250, 17])

        batch_size, num_queries, _ = pred_logits.shape
        criterion = nn.CrossEntropyLoss(weight=self.empty_weight)
        # INFO empty_weight = torch.ones(self.num_classes + 1) and empty_weight[-1] = self.eos_coef=0.1
        idx = self._get_predictions_permutation_indices(indices) #batch_indices, queries_indices

        target_classes_o = torch.cat([target[j] for target, (_, j) in zip(class_labels, indices)]) # shape = (batch_size, num_queries)
        target_classes = torch.full((batch_size, num_queries), fill_value=self.num_classes, dtype=torch.int64, device=pred_logits.device) # shape = (batch_size, num_queries)
        target_classes[idx] = target_classes_o
        print(f"target_classes: {target_classes.shape}, {target_classes}") #torch.Size([2, 250])

        # permute pred_logits (batch_size, num_queries, num_labels) -> (batch_size, num_labels, num_queries)
        pred_logits_transposed = pred_logits.transpose(1, 2)
        print(f"pred_logits_transposed: {pred_logits_transposed.shape}") # torch.Size([2, 17, 250])

        loss_ce = criterion(pred_logits_transposed, target_classes)
        print(f"loss_cross_entropy loss_ce: {loss_ce}") # tensor(2.5460)

        # New - so we need to implement ohem here for masks 250 batchwise based on threshold, minkept, also class importance
        if self.class_specific_weights is not None:
            ce_class_weights = self.class_specific_weights
        else:
            ce_class_weights = self.empty_weight
        loss_manual_no_reduction = nn.CrossEntropyLoss(weight=ce_class_weights, reduction="none") # new to get losses per query
        loss_ce_without_reduction = loss_manual_no_reduction(pred_logits_transposed, target_classes)
        print(f"loss_ce_without_reduction: {loss_ce_without_reduction.shape}, {loss_ce_without_reduction}") # torch.Size([2, 250])
        loss_ce_manual_mean = loss_ce_without_reduction.sum() / self.empty_weight[target_classes].sum() # this is equal to usual weighted ce loss with reduction
        print(f"loss_cross_entropy mean loss_ce_manual: {loss_ce_manual_mean}") # tensor(2.5460)

        # losses = {"loss_cross_entropy": loss_ce}
        losses = {"loss_cross_entropy": loss_ce_manual_mean} #new
        print(f"loss_cross_entropy losses: {losses}") #{'loss_cross_entropy': tensor(2.5460, device='cuda:0', grad_fn=<NllLoss2DBackward0>)}
        return losses

    def loss_masks(self, masks_queries_logits: Tensor, mask_labels: List[Tensor], indices: Tuple[np.array], num_masks: int) -> Dict[str, Tensor]:
        """
            masks_queries_logits `batch_size, num_queries, height, width`
            mask_labels :List of mask labels of shape `(labels, height, width)`.
            indices (`Tuple[np.array])`:
            num_masks (`int)`: The number of masks, used for normalization.
        """
        src_idx = self._get_predictions_permutation_indices(indices)
        tgt_idx = self._get_targets_permutation_indices(indices)
        print(f"src_idx: {src_idx}, tgt_idx: {tgt_idx}")
        #src_idx: (tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), tensor([  2,  29,  39, 182, 194, 249,   0,  23,  29,  51, 160, 178, 204, 211, 215, 236, 249])),
        #tgt_idx: (tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), tensor([ 0,  5,  1,  3,  2,  4,  0,  7,  3,  5,  1,  2,  4,  6, 10,  8,  9]))

        # masks_queries_logits - shape (batch_size, num_queries, height, width) = (2, 250, 128, 128)
        pred_masks = masks_queries_logits[src_idx]

        # shape (batch_size, num_queries, height, width)
        # pad all and stack the targets to the num_masks dimension
        # upsample predictions to the target size, we have to add one dim to use interpolate
        target_masks, _ = self._pad_images_to_max_in_batch(mask_labels)
        target_masks = target_masks[tgt_idx]

        print(f"loss_masks: pred_masks- {pred_masks.shape}, target_masks- {target_masks.shape}")
        #loss_masks: pred_masks- torch.Size([17, 128, 128]), target_masks- torch.Size([17, 512, 512])

        pred_masks = pred_masks[:, None] # a new axis is added to the pred_masks tensor. Specifically, a singleton dimension (dimension of size 1) is inserted at the second position (index 1) of the tensor's shape.
        target_masks = target_masks[:, None]
        print(f"pred_masks- {pred_masks.shape}, target_masks- {target_masks.shape}")
        # pred_masks- torch.Size([17, 1, 128, 128]), target_masks- torch.Size([17, 1, 512, 512])
        with torch.no_grad():
            # sample point_coords - PointRend (mask2former)
            point_coords = self.sample_points_using_uncertainty(
                pred_masks, #torch.Size([17, 1, 128, 128])
                self.calculate_uncertainty, #uncertainty function
                self.num_points, #12544
                self.oversample_ratio, #3.0
                self.importance_sample_ratio, #0.75
            )
            print(f"point_coords- {point_coords.shape}") # torch.Size([17, 12544, 2]) , where K = num_points sampled = 12544 = 112x112
            # get ground-truth labels
            point_labels = sample_point(target_masks, point_coords, align_corners=False).squeeze(1)
            print(f"point_labels: {point_labels.shape}") #point_labels: torch.Size([17, 12544]),
        point_logits = sample_point(pred_masks, point_coords, align_corners=False).squeeze(1)
        print(f"point_logits: {point_logits.shape}") # point_logits: torch.Size([17, 12544])
        print(f"num_masks: {num_masks}") # num_masks: tensor([17.])

        losses = {
            "loss_mask": sigmoid_cross_entropy_loss(point_logits, point_labels, num_masks),
            # "loss_mask": sigmoid_cross_entropy_loss(pred_masks.squeeze(), target_masks.squeeze(), num_masks),
            "loss_dice": dice_loss(point_logits, point_labels, num_masks),
        }
        print(f"loss_masks losses: {losses}")
        del pred_masks
        del target_masks
        return losses

    def loss_distillation(self, teacher_queries_distillation_logits: Tensor, student_queries_distillation_logits: Tensor):  # new
        student_queries_distillation_logits = student_queries_distillation_logits.float() #bs, 234, 512
        # print("loss_distillation student_queries_distillation_logits, teacher_queries_distillation_logits")
        # print(student_queries_distillation_logits.shape)
        # print(teacher_queries_distillation_logits.shape)
        # print(torch.unique(torch.eq(student_queries_distillation_logits, teacher_queries_distillation_logits)))
        # normalize
        teacher_logits = nn.functional.normalize(teacher_queries_distillation_logits.flatten(1), dim=-1) # FixMe: Do we need for logit scale?
        student_logits = nn.functional.normalize(student_queries_distillation_logits.flatten(1), dim=-1)
        loss_distillation = nn.functional.mse_loss(student_logits, teacher_logits) # FixMe: Get the criterion from args later,, we huber loss, or l1 loss or mse loss
        losses = {"loss_distillation": loss_distillation}
        return losses

    def forward(self, masks_queries_logits: Tensor, class_queries_logits: Tensor, contrastive_queries_logits: Tensor, mask_labels: List[Tensor],
                class_labels: List[Tensor], text_queries: Tensor, teacher_queries_distillation_logits: Tensor,
                student_queries_distillation_logits: Tensor, auxiliary_predictions: Optional[Dict[str, Tensor]] = None, calculate_contrastive_loss: bool = True,
        ) -> Dict[str, Tensor]:

        # retrieve the matching between the outputs of the last layer and the labels
        indices = self.matcher(masks_queries_logits, class_queries_logits, mask_labels, class_labels)
        # compute the average number of target masks for normalization purposes
        num_masks = self.get_num_masks(class_labels, device=class_labels[0].device)
        # get all the losses
        losses: Dict[str, Tensor] = {
            **self.loss_masks(masks_queries_logits, mask_labels, indices, num_masks),
            **self.loss_labels(class_queries_logits, class_labels, indices),
        }
        if calculate_contrastive_loss:
            losses = {
                **losses,
                **self.loss_contrastive(contrastive_queries_logits, text_queries),
                **self.loss_distillation(teacher_queries_distillation_logits, student_queries_distillation_logits)  # new
            }

        # in case of auxiliary losses, we repeat this process with the output of each intermediate layer.
        if auxiliary_predictions is not None:
            for idx, aux_outputs in enumerate(auxiliary_predictions):
                masks_queries_logits = aux_outputs["masks_queries_logits"]
                class_queries_logits = aux_outputs["class_queries_logits"]
                loss_dict = self.forward(
                    masks_queries_logits,
                    class_queries_logits,
                    None,
                    mask_labels,
                    class_labels,
                    None,
                    None,
                    None,  # new
                    calculate_contrastive_loss=False,
                )
                loss_dict = {f"{key}_{idx}": value for key, value in loss_dict.items()}
                print("auxiliary losses")
                print(len(auxiliary_predictions)) #9
                print(len(loss_dict)) #3
                losses.update(loss_dict)

        return losses

``` self explanation of loss_masks
#masks_queries_logits `batch_size, num_queries, height, width`

import torch
masks_queries_logits = torch.randn(size=(2, 250, 128, 128))
masks_queries_logits.shape
src_idx = (torch.tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
           torch.tensor([  2,  29,  39, 182, 194, 249,   0,  23,  29,  51, 160, 178, 204, 211, 215, 236, 249]))
tgt_idx = (torch.tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
           torch.tensor([ 0,  5,  1,  3,  2,  4,  0,  7,  3,  5,  1,  2,  4,  6, 10,  8,  9]))
        
# shape (batch_size * num_queries, height, width)
pred_masks = masks_queries_logits[src_idx]
pred_masks.shape #torch.Size([17, 128, 128])
```

In [32]:
from transformers.models.oneformer.modeling_oneformer import OneFormerForUniversalSegmentation
from transformers import CLIPTextModel

class CustomOneFormerForUniversalSegmentation(OneFormerForUniversalSegmentation):

    def __init__(self, config):
        super().__init__(config)
        self.model = CustomOneFormerModel(config)
        self.is_training = config.is_training
        if self.is_training:
            print("Initliazing the pretrained teacher model")
            self.teacher_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
        else:
            self.teacher_model = None

        self.weight_dict: Dict[str, float] = {
             "loss_cross_entropy": config.class_weight, "loss_mask": config.mask_weight, "loss_dice": config.dice_weight,
             "loss_contrastive": config.contrastive_weight, "loss_distillation": config.contrastive_weight,  # new # FIX ME: Setting the weight for distillation loss later. For now, we are using same weight as constrastive weight = 0.5
        }
        if "class_specific_weights" in config:
            class_specific_weights = config.class_specific_weights
        else:
            class_specific_weights = None

        self.criterion = CustomOneFormerLoss(
            class_specific_weights=class_specific_weights, #new for class weights
            num_classes=config.num_labels, matcher=self.matcher, weight_dict=self.weight_dict, eos_coef=config.no_object_weight, num_points=config.train_num_points,
            oversample_ratio=config.oversample_ratio, importance_sample_ratio=config.importance_sample_ratio, contrastive_temperature=config.contrastive_temperature,
        )

    def get_loss_dict(self, masks_queries_logits: Tensor, class_queries_logits: Tensor, contrastive_queries_logits: Tensor, mask_labels: Tensor,
                      class_labels: Tensor, text_queries: Tensor, auxiliary_predictions: Dict[str, Tensor], teacher_queries_distillation_logits: Tensor,
                      student_queries_distillation_logits: Tensor, calculate_contrastive_loss: bool,
        ) -> Dict[str, Tensor]:
        loss_dict: Dict[str, Tensor] = self.criterion(
            masks_queries_logits=masks_queries_logits, class_queries_logits=class_queries_logits, contrastive_queries_logits=contrastive_queries_logits,
            mask_labels=mask_labels, class_labels=class_labels, text_queries=text_queries, teacher_queries_distillation_logits=teacher_queries_distillation_logits,
            student_queries_distillation_logits=student_queries_distillation_logits,
            auxiliary_predictions=auxiliary_predictions, calculate_contrastive_loss=calculate_contrastive_loss,
        )  # new
        # weight each loss by `self.weight_dict[<LOSS_NAME>]` including auxiliary losses
        print("loss weights")
        print(f"weight_dict, {self.weight_dict.items()}")
        # weight_dict, dict_items([('loss_cross_entropy', 2.0), ('loss_mask', 5.0), ('loss_dice', 5.0), ('loss_contrastive', 0.5), ('loss_distillation', 0.5)])
        print(f"loss_dict: {loss_dict.items()}")
        ##loss_dict: dict_items(
        # batch mean losses [('loss_mask', tensor([0.5695], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_dice', tensor([0.4705], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_cross_entropy', tensor(2.6274, device='cuda:0', grad_fn=<NllLoss2DBackward0>)), ('loss_contrastive', tensor(0., device='cuda:0', grad_fn=<AddBackward0>)), ('loss_distillation', tensor(1.7493e-05, device='cuda:0', grad_fn=<MseLossBackward0>)),
        # auxiliary losses: ('loss_mask_0', tensor([0.4521], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_dice_0', tensor([0.5151], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_cross_entropy_0', tensor(2.6126, device='cuda:0', grad_fn=<NllLoss2DBackward0>)),
        # ('loss_mask_1', tensor([0.5129], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_dice_1', tensor([0.5169], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_cross_entropy_1', tensor(2.8194, device='cuda:0', grad_fn=<NllLoss2DBackward0>)),
        # ('loss_mask_2', tensor([0.6444], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_dice_2', tensor([0.4852], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_cross_entropy_2', tensor(2.6235, device='cuda:0', grad_fn=<NllLoss2DBackward0>)),
        # ('loss_mask_3', tensor([0.5162], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_dice_3', tensor([0.4862], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_cross_entropy_3', tensor(2.5991, device='cuda:0', grad_fn=<NllLoss2DBackward0>)),
        # ('loss_mask_4', tensor([0.5660], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_dice_4', tensor([0.5032], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_cross_entropy_4', tensor(2.6130, device='cuda:0', grad_fn=<NllLoss2DBackward0>)),
        # ('loss_mask_5', tensor([0.5792], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_dice_5', tensor([0.5023], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_cross_entropy_5', tensor(2.7068, device='cuda:0', grad_fn=<NllLoss2DBackward0>)),
        # ('loss_mask_6', tensor([0.6443], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_dice_6', tensor([0.5041], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_cross_entropy_6', tensor(2.6910, device='cuda:0', grad_fn=<NllLoss2DBackward0>)),
        # ('loss_mask_7', tensor([0.6240], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_dice_7', tensor([0.5005], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_cross_entropy_7', tensor(2.6339, device='cuda:0', grad_fn=<NllLoss2DBackward0>)),
        # ('loss_mask_8', tensor([0.5074], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_dice_8', tensor([0.4951], device='cuda:0', grad_fn=<DivBackward0>)), ('loss_cross_entropy_8', tensor(2.5680, device='cuda:0', grad_fn=<NllLoss2DBackward0>))])
        for key, weight in self.weight_dict.items():
            for loss_key, loss in loss_dict.items():
                if key in loss_key:
                    print(loss_key, loss, weight)
                    loss *= weight
        return loss_dict

    def get_loss(self, loss_dict: Dict[str, Tensor]) -> Tensor: #ohem
        # if use_ohem: # prepare someflag to enable ohem

        return sum(loss_dict.values())

    def get_teacher_logits(self, teacher_text_inputs):  # new
        if teacher_text_inputs.ndim is None:
            raise ValueError("text must not be NoneType")
        if teacher_text_inputs.ndim not in [2, 3]:
            raise ValueError("Number of dimensions in text must be 2 or 3")
        batch_size, num_text, hidden_dim_seq = teacher_text_inputs.shape
        teacher_text_inputs = teacher_text_inputs.reshape(batch_size * num_text, hidden_dim_seq)
        self.teacher_model.to(self.model.device)
        self.teacher_model.eval()
        with torch.inference_mode():
            print("teacher logits generation")
            teacher_text_queries_distillation_logits = self.teacher_model(teacher_text_inputs)
            _, hidden_dim = teacher_text_queries_distillation_logits["pooler_output"].shape
            teacher_text_queries_distillation_logits = teacher_text_queries_distillation_logits["pooler_output"].reshape(batch_size, num_text, hidden_dim).float() #bs, 234, 512
            print("teacher_text_queries_distillation_logits shape")
            print(teacher_text_queries_distillation_logits.shape)
        return teacher_text_queries_distillation_logits

    def forward(self, pixel_values: Tensor, task_inputs: Tensor, text_inputs: Optional[Tensor] = None, mask_labels: Optional[List[Tensor]] = None,
                 class_labels: Optional[List[Tensor]] = None, pixel_mask: Optional[Tensor] = None, output_auxiliary_logits: Optional[bool] = None,
                 output_hidden_states: Optional[bool] = None, output_attentions: Optional[bool] = None, return_dict: Optional[bool] = None,
        ) -> CustomOneFormerForUniversalSegmentationOutput:
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states)
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.model(pixel_values=pixel_values, task_inputs=task_inputs, text_inputs=text_inputs, pixel_mask=pixel_mask,
                             output_hidden_states=output_hidden_states or self.config.use_auxiliary_loss,
                             output_attentions=output_attentions, return_dict=True,)

        loss, loss_dict, auxiliary_predictions = None, None, None
        class_queries_logits = outputs.transformer_decoder_class_predictions
        masks_queries_logits = outputs.transformer_decoder_mask_predictions
        contrastive_queries_logits = outputs.transformer_decoder_contrastive_queries
        auxiliary_predictions = outputs.transformer_decoder_auxiliary_predictions
        text_queries = outputs.text_queries

        if self.is_training:  # new
            teacher_text_queries_distillation_logits = self.get_teacher_logits(text_inputs)
            student_text_queries_distillation_logits = outputs["text_queries_distillation_logits"]
        else:
            teacher_text_queries_distillation_logits, student_text_queries_distillation_logits = None, None

        if mask_labels is not None and class_labels is not None:
            loss_dict: Dict[str, Tensor] = self.get_loss_dict(masks_queries_logits=masks_queries_logits, class_queries_logits=class_queries_logits,
                                                              contrastive_queries_logits=contrastive_queries_logits, mask_labels=mask_labels,
                                                              class_labels=class_labels, text_queries=text_queries, auxiliary_predictions=auxiliary_predictions,
                                                              teacher_queries_distillation_logits=teacher_text_queries_distillation_logits,  # new
                                                              student_queries_distillation_logits=student_text_queries_distillation_logits,
                                                              calculate_contrastive_loss=self.config.contrastive_temperature is not None,
                                                              )
            loss = self.get_loss(loss_dict)

        output_auxiliary_logits = (self.config.output_auxiliary_logits if output_auxiliary_logits is None else output_auxiliary_logits)
        if not output_auxiliary_logits:
            auxiliary_predictions = None

        output = CustomOneFormerForUniversalSegmentationOutput(class_queries_logits=class_queries_logits, masks_queries_logits=masks_queries_logits,
                                                               auxiliary_predictions=auxiliary_predictions, loss=loss, **outputs,)
        if not return_dict:
            output = tuple(v for v in output.values())
            if loss is not None:
                output = (loss) + output
        return output

```
'no_object_weight': 0.1,
 'class_weight': 2.0,
 'mask_weight': 5.0,
 'dice_weight': 5.0,
 'contrastive_weight': 0.5,
 'contrastive_temperature': 0.07,
 'train_num_points': 12544,
 'oversample_ratio': 3.0,
  'importance_sample_ratio': 0.75,
 ```

In [40]:
from transformers import AutoProcessor, AutoModelForUniversalSegmentation, OneFormerForUniversalSegmentation, AutoConfig, CLIPTextModel
import torch.nn.functional as F
from torch import nn
import gc

# # Processor
# model_ckpt = "shi-labs/oneformer_cityscapes_swin_large"
# processor = AutoProcessor.from_pretrained(model_ckpt, ignore_index=255, do_reduce_labels=False)
# processor.image_processor.num_text = 234
# model_config = AutoConfig.from_pretrained(model_ckpt, text_encoder_width=512, text_encoder_num_layers=2, is_training=True)
# model = CustomOneFormerForUniversalSegmentation.from_pretrained(model_ckpt, ignore_mismatched_sizes=True, config=model_config)

# id2label
id2label, label2id, labels = get_modified_labels()
# model and processor
model_ckpt = "shi-labs/oneformer_cityscapes_swin_large"
model_config = AutoConfig.from_pretrained(
    model_ckpt, text_encoder_width=512, text_encoder_num_layers=2, is_training=True, id2label=id2label, label2id=label2id, num_labels=len(id2label),
    use_auxiliary_loss=False, output_auxiliary_logits=False
)
model = CustomOneFormerForUniversalSegmentation.from_pretrained(model_ckpt, ignore_mismatched_sizes=True, config=model_config)
# Freeze the model backbone params
for name, params in model.named_parameters():
    if name.startswith("model.pixel_level_module"):
        params.requires_grad = False
processor = AutoProcessor.from_pretrained(model_ckpt, ignore_index=255, do_reduce_labels=False, do_rescale=False, do_resize=False)
metadata_id2label = {str(k): v for k, v in id2label.items()}
metadata = deepcopy(metadata_id2label)
metadata["thing_ids"] = [11, 12, 13, 14]
metadata["class_names"] = labels
processor.image_processor.metadata = metadata
processor.image_processor.num_labels = len(id2label)
processor.image_processor.num_text = model.config.num_queries - model.config.text_encoder_n_ctx

#dataset
mydataset = load_railsem_dataset()
total_size = len(mydataset["data"])
print(f"[INFO]: Total images: {total_size}")
train_size = 8160
train_percentage = train_size / total_size
# Use train_test_split with specified percentages
splits = mydataset["data"].train_test_split(train_size=train_percentage, shuffle=True)
train_split = splits["train"]
val_split = splits["test"].select(indices=(range(320))) #320
test_split = splits["test"].select(indices=(range(20)))
print(f"[INFO]: Total Training images: {len(train_split)}")
print(f"[INFO]: Total Validation images: {len(val_split)}")
print(f"[INFO]: Total Test images: {len(test_split)}")
# transforms
transforms = get_transforms()
customdataset = CustomDataset(train_split.select(range(0, 6)), transforms)
patch_size = [512, 512]
train_queue = PatchQueue(customdataset, max_length=10, samples_per_image=5, patch_size=patch_size)
dataloader = _prepare_dataloader(train_queue)
device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.cuda.is_available():
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.empty_cache()
    gc.collect()
model.to(device)

#model.eval()
model.train()
for step, batch in enumerate(dataloader):
    print(f"Step: {step}")
    batch_dict = {
            "pixel_values": batch["pixel_values"].to(device),
            "mask_labels": [labels.to(device)
                            for labels in batch["mask_labels"]],
            "class_labels": [labels.to(device)
                                for labels in batch["class_labels"]],
            "pixel_mask": batch["pixel_mask"].to(device),
            "text_inputs": batch["text_inputs"].to(device),
            "task_inputs": batch["task_inputs"].to(device),
    }
    #with torch.no_grad():
    with torch.autocast(device_type='cuda', dtype=torch.float16):
        print(f'mask_labels: {batch["mask_labels"][0].shape}, {batch["mask_labels"][1].shape}') # torch.Size([6, 512, 512]), torch.Size([11, 512, 512])
        print(f'class_labels: {batch["class_labels"][0]}, {batch["class_labels"][1]}') # class_labels: tensor([ 2,  4,  5,  8,  9, 10]), tensor([ 0,  1,  2,  3,  4,  5,  7,  9, 10, 11, 12])
        outputs = model(**batch_dict)
        print(outputs.keys())
        print(f'class_queries_logits: {outputs["class_queries_logits"].shape}')
        print(f'masks_queries_logits: {outputs["masks_queries_logits"].shape}')
        loss = outputs.loss
        print("final loss")
        print(loss)
    loss.backward()
    del batch
    break

Initliazing the pretrained teacher model


Some weights of CustomOneFormerForUniversalSegmentation were not initialized from the model checkpoint at shi-labs/oneformer_cityscapes_swin_large and are newly initialized: ['model.text_mapper.prompt_ctx.weight', 'model.text_mapper.text_encoder.ln_final.bias', 'model.text_mapper.text_encoder.ln_final.weight', 'model.text_mapper.text_encoder.positional_embedding', 'model.text_mapper.text_encoder.token_embedding.weight', 'model.text_mapper.text_encoder.transformer.layers.0.layer_norm1.bias', 'model.text_mapper.text_encoder.transformer.layers.0.layer_norm1.weight', 'model.text_mapper.text_encoder.transformer.layers.0.layer_norm2.bias', 'model.text_mapper.text_encoder.transformer.layers.0.layer_norm2.weight', 'model.text_mapper.text_encoder.transformer.layers.0.mlp.fc1.bias', 'model.text_mapper.text_encoder.transformer.layers.0.mlp.fc1.weight', 'model.text_mapper.text_encoder.transformer.layers.0.mlp.fc2.bias', 'model.text_mapper.text_encoder.transformer.layers.0.mlp.fc2.weight', 'model.t

[INFO]: Extracting Railsem19 dataset from hub on
[INFO]: Total images: 8500
[INFO]: Total Training images: 8160
[INFO]: Total Validation images: 320
[INFO]: Total Test images: 20
[INFO]: Found labels_info.json.Skipping Download...
[WARN]: Patch List is empty...
[WARN]:Exception while getting image for patching: 'NoneType' object is not an iterator
Step: 0
mask_labels: torch.Size([4, 512, 512]), torch.Size([10, 512, 512])
class_labels: tensor([ 2,  4,  5, 10]), tensor([ 0,  1,  2,  3,  4,  7,  8, 10, 11, 12])
inside my custom model
custom text mapper encoder output
text shape in encode_text textmapper- (bs, num_text, hidden_dim)
torch.Size([2, 234, 77])
encoded_text
torch.Size([468, 512])
text_queries_before_ctx
torch.Size([2, 234, 512])
text_queries after projection
torch.Size([468, 256])
teacher logits generation
teacher_text_queries_distillation_logits shape
torch.Size([2, 234, 512])
loss_masks: pred_masks- torch.Size([14, 128, 128]), target_masks- torch.Size([14, 512, 512])
src_idx:

In [ ]:
len(dataloader), len(train_queue)

In [ ]:
from transformers import AutoProcessor, AutoModelForUniversalSegmentation, OneFormerForUniversalSegmentation, AutoConfig, CLIPTextModel
import torch.nn.functional as F
from torch import nn

customdataset = CustomDataset(test_split.select(range(0, 6)))
testdataloader = _prepare_dataloader(customdataset)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Processor
model_ckpt = "shi-labs/oneformer_cityscapes_swin_large"
processor = AutoProcessor.from_pretrained(model_ckpt, ignore_index=255, do_reduce_labels=False)
processor.image_processor.num_text = 234
model_config = AutoConfig.from_pretrained(model_ckpt, text_encoder_width=512, text_encoder_num_layers=2, is_training=True)
model = CustomOneFormerForUniversalSegmentation.from_pretrained(model_ckpt, ignore_mismatched_sizes=True, config=model_config)
teacher_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
teacher_model.to(model.device)
teacher_model.eval()
distillation_criterion = F.mse_loss # FixMe: Get the criterion from args later,, we huber loss, or l1 loss or mse loss
distillation_weight = 0.5
distillation_temperature = 0.07
# logit_scale = nn.Parameter(torch.tensor(np.log(1 / distillation_temperature)))

def get_features(dataset):
    all_features = []
    teacher_model.eval()
    with torch.no_grad():
        for step, batch in enumerate(testdataloader):
            print(batch["text_inputs"][0].shape, batch["text_inputs"][1].shape) #each text embedding is of torch.Size([234, 77])(num_queries, seqlength) => whole batch is (2, 234, 77),
            #concatenate the batchdimension
            text = batch["text_inputs"]
            batch_size, num_text, hidden_dim_seq = text.shape
            text = text.reshape(batch_size * num_text, hidden_dim_seq)

            # take features from the eot embedding (eot_token is the highest number in each sequence)
            text_queries = teacher_model(text)
            print(text_queries.keys())
            print(text_queries["last_hidden_state"].shape) #torch.Size([234, 77, 512])  #in clip  # text_embeds.shape = [batch_size, sequence_length, transformer.width], but we have 234 sequences considering as batchsize for each ex
            print(text_queries["text_embeds"].shape) #pooler_output for withoutprojection #torch.Size([234, 512])
            _, hidden_dim = text_queries["text_embeds"].shape
            text_queries = text_queries["text_embeds"].reshape(batch_size, num_text, hidden_dim)
            # our actual model has  [batch_size, 234, 256]
            # text_features = [teacher_model(example) for example in batch["text_inputs"]] #text_inputs contains input_ids with attention_mask multiplied by imageprocesor of oneformer, we need texts to tokenize
            # all_features.extend(text_features)
            del batch
            break
        #, text_features
    return  text_queries# torch.stack(all_features), all_features

model.eval()
for step, batch in enumerate(testdataloader):
    batch_dict = {
            "pixel_values": batch["pixel_values"].to(device),
            "mask_labels": [labels.to(device)
                            for labels in batch["mask_labels"]],
            "class_labels": [labels.to(device)
                                for labels in batch["class_labels"]],
            "pixel_mask": batch["pixel_mask"].to(device),
            "text_inputs": batch["text_inputs"].to(device),
            "task_inputs": batch["task_inputs"].to(device),
    }
    text = batch["text_inputs"]
    batch_size, num_text, hidden_dim_seq = text.shape
    text = text.reshape(batch_size * num_text, hidden_dim_seq)
    #with torch.autocast():
    with torch.no_grad():
        outputs = model(**batch_dict)
        print(outputs.keys())
        loss = outputs.loss
        print(loss)
    with torch.inference_mode():
        print("teacher")
        teacher_queries_logits = teacher_model(text)
        _, hidden_dim = teacher_queries_logits["pooler_output"].shape
        teacher_queries_logits = teacher_queries_logits["pooler_output"].reshape(batch_size, num_text, hidden_dim).float() #bs, 234, 512
        student_queries_logits = outputs["text_queries_distillation_logits"].float() #bs, 234, 512
        # normalize
        teacher_queries_logits = nn.functional.normalize(teacher_queries_logits.flatten(1), dim=-1) #check whether the embeddings are normalixing already and need for logit scale
        student_queries_logits = nn.functional.normalize(student_queries_logits.flatten(1), dim=-1)
        ## logit scale clamp
        # logit_scale = torch.clamp(logit_scale.exp(), max=100)
        #teacher_queries_logits = teacher_queries_logits * logit_scale
        # student_queries_logits = student_queries_logits * logit_scale
        distill_loss = distillation_criterion(student_queries_logits, teacher_queries_logits)
        print(distill_loss)
        loss = distill_loss * distillation_weight + loss
        print(loss)
    # loss.backward()

    del batch
    break


In [ ]:
outputs["text_queries_distillation_logits"].shape, outputs["text_queries"].shape, outputs["transformer_decoder_contrastive_queries"].shape

In [ ]:
#create in oneformertrainer - check the config is_training
def distill_oneformertextencoder():
    model_ckpt = "shi-labs/oneformer_cityscapes_swin_large"
    processor = AutoProcessor.from_pretrained(model_ckpt, ignore_index=255, do_reduce_labels=False)
    processor.image_processor.num_text = 234
    model_config = AutoConfig.from_pretrained(model_ckpt, text_encoder_width=512, text_encoder_num_layers=2, is_training=True)
    student_model = CustomOneFormerForUniversalSegmentation.from_pretrained(model_ckpt, ignore_mismatched_sizes=True, config=model_config)
    return student_model

## Distllator class
class TextDistillator(nn.Module):
    #self, teacher_model=None, student_model=None, temperature=None, lambda_param=None,  *args, **kwargs
    def __init__(self, teacher_model, student_model, temperature: float = 1.0, training=False) -> None:
        super(TextDistillator, self).__init__()
        self.student = student_model
        self.training = student_model.config.is_training
        self.temperature = temperature
        self.teacher = teacher_model #clip model
        # place teacher on same device as student
        self.teacher.to(self.student.device)
        self.teacher.eval()

    @property
    def temperature(self) -> float:
        return self._temperature if self.training else 1 #1 for inference

    @temperature.setter
    def temperature(self, value : float) -> None:
        if value < 1:
            raise(ValueError(f"Temperature must be above 1, it cannot be {value}"))
        self._temperature = value

    def get_logits(self, input_ids: Tensor=None, attention_mask: Tensor=None, from_teacher: bool = False) -> Tensor:
        if from_teacher:
            return self.teacher.classifier(self.roberta(input_ids, attention_mask)[0])
        return self.student.classifier(self.student(input_ids, attention_mask)[0])

    def forward(self, batch_dict : Tensor) -> Tuple[Tensor, Tensor]:
        student_ouputs = self.get_logits(**batch_dict)
        student_text_encoder_queries_logits = student_ouputs["text_queries"]
        teacher_text_queries_embeddings = batch_dict["text_inputs"]
        teacher_logits = self.get_logits()
        return student_logits.softmax(1), self.loss(teacher_logits, student_logits, labels)

    def compute_loss(self, teacher_logits, student_logits, return_outputs=False):
        student_output = self.student(**inputs)

        with torch.no_grad():
          teacher_output = self.teacher(**inputs)

        # Compute soft targets for teacher and student
        soft_teacher = F.softmax(teacher_output.logits / self.temperature, dim=-1)
        soft_student = F.log_softmax(student_output.logits / self.temperature, dim=-1)

        # Compute the loss
        distillation_loss = self.loss_function(soft_student, soft_teacher) * (self.temperature ** 2)

        # Compute the true label loss
        student_target_loss = student_output.loss

        # Calculate final loss
        loss = (1. - self.lambda_param) * student_target_loss + self.lambda_param * distillation_loss
        return (loss, student_output) if return_outputs else loss

    # def loss(self,
    #     teacher_logits : Tensor,
    #     student_logits : Tensor,
    #     labels : Tensor,
    # ) -> Tensor:
    #     """
    #     The distillation loss for distilating a BERT-like model.
    #     The loss takes the (teacher_logits), (student_logits) and (labels) for various losses.
    #     """
    #     # Temperature and sotfmax
    #     student_logits, teacher_logits = (student_logits / self.temperature).softmax(1), (teacher_logits / self.temperature).softmax(1)
    #     # Classification loss (problem-specific loss)
    #     loss = CrossEntropyLoss()(student_logits, labels)
    #     # CrossEntropy teacher-student loss
    #     loss = loss + CrossEntropyLoss()(student_logits, teacher_logits)
    #     # Cosine loss
    #     loss = loss + CosineEmbeddingLoss()(teacher_logits, student_logits, torch.ones(teacher_logits.size()[0]))
    #     # Average the loss and return it
    #     loss = loss / 3
    #     return loss

In [ ]:
from transformers import CLIPTextModel, CLIPTextModelWithProjection, CLIPTextConfig

#teacher_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
teacher_config = CLIPTextConfig.from_pretrained("openai/clip-vit-base-patch32")
# teacher_config.projection_dim = 256  # RuntimeError: Error(s) in loading state_dict for CLIPTextModelWithProjection: size mismatch for text_projection.weight: copying a param with shape torch.Size([512, 512]) from checkpoint, the shape in current model is torch.Size([256, 512]). You may consider adding `ignore_mismatched_sizes=True` in the model `from_pretrained` method.
teacher_model = CLIPTextModelWithProjection.from_pretrained("openai/clip-vit-base-patch32", config=teacher_config)

customdataset = CustomDataset(test_split.select(range(0, 6)))
testdataloader = _prepare_dataloader(customdataset)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(len(testdataloader))

def get_features(dataset):
    all_features = []
    teacher_model.eval()
    with torch.no_grad():
        for step, batch in enumerate(testdataloader):
            print(batch["text_inputs"][0].shape, batch["text_inputs"][1].shape) #each text embedding is of torch.Size([234, 77])(num_queries, seqlength) => whole batch is (2, 234, 77),
            #concatenate the batchdimension
            text = batch["text_inputs"]
            batch_size, num_text, hidden_dim_seq = text.shape
            text = text.reshape(batch_size * num_text, hidden_dim_seq)

            # take features from the eot embedding (eot_token is the highest number in each sequence)
            text_queries = teacher_model(text)
            print(text_queries.keys())
            print(text_queries["last_hidden_state"].shape) #torch.Size([234, 77, 512])  #in clip  # text_embeds.shape = [batch_size, sequence_length, transformer.width], but we have 234 sequences considering as batchsize for each ex
            print(text_queries["text_embeds"].shape) #pooler_output for withoutprojection #torch.Size([234, 512])
            _, hidden_dim = text_queries["text_embeds"].shape
            text_queries = text_queries["text_embeds"].reshape(batch_size, num_text, hidden_dim)
            # our actual model has  [batch_size, 234, 256]
            # text_features = [teacher_model(example) for example in batch["text_inputs"]] #text_inputs contains input_ids with attention_mask multiplied by imageprocesor of oneformer, we need texts to tokenize
            # all_features.extend(text_features)
            del batch
            break
        #, text_features
    return  text_queries# torch.stack(all_features), all_features

text_queries = get_features(customdataset)

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import clip

customdataset = CustomDataset(test_split.select(range(0, 5)))
testdataloader = _prepare_dataloader(customdataset)


device = "cuda" if torch.cuda.is_available() else "cpu"
teacher_model, preprocess = clip.load("ViT-B/32", device=device)
teacher_model.eval()
# for step, batch in enumerate(testdataloader):
#     print(batch.keys())
#     print(batch["text_inputs"].shape) #[2, 234, 77]
#     print(batch["pixel_values"].shape) #[2, 3, 1080, 1920]
#     with torch.no_grad():
#         #teacher_outputs = teacher_model(batch["pixel_values"], batch["text_inputs"])
#         text_features = teacher_model.encode_text(batch["text_inputs"].squeeze())
#         text_features.shape # our actual model has  [batch_size, 234, 256]
#         break

def get_features(dataset):
    all_features = []
    with torch.no_grad():
        for step, batch in enumerate(testdataloader):
            # outputs = teacher_model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
            text_features = [teacher_model.encode_text(example) for example in batch["text_inputs"]]
            all_features.extend(text_features)
            del batch, text_features
    return torch.stack(all_features), all_features

# Calculate the image features
train_features, all_features = get_features(customdataset)



#image.shape, text.shape, text_features.shape, image_features.shape
# (torch.Size([1, 3, 224, 224]),
#  torch.Size([3, 77]),
#  torch.Size([1, 3, 512]),
#  torch.Size([1, 1, 512]))

In [ ]:
train_features.shape, len(all_features), all_features[0].shape #len(all_features)

In [ ]:
torch.stack(train_features[1]).shape

Note: we can get text lits from oneformer imageprocessor

In [ ]:
from transformers import CLIPTokenizer, CLIPTextModel

tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
teacher_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
teacher_model.eval()

inputs = tokenizer(["a photo of a cat"]*20, padding="max_length", max_length=77, truncation=True, return_tensors="pt")
print(inputs.keys())
print(inputs["input_ids"].shape)
print(inputs["attention_mask"].shape)


# adding batch dimension
inputs["input_ids"] = inputs["input_ids"].unsqueeze(dim=0)
inputs["attention_mask"]= inputs["attention_mask"].unsqueeze(dim=0)
print(inputs["input_ids"].shape)
print(inputs["attention_mask"].shape)
outputs = teacher_model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
print(outputs.keys(), outputs["last_hidden_state"].shape, outputs["pooler_output"].shape)


In [ ]:
teacher_model.config

In [ ]:
teacher_model

In [ ]:
torchinfo.summary(teacher_model)

In [ ]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

#image = preprocess(Image.open("CLIP.png")).unsqueeze(0).to(device)
text = clip.tokenize(["a diagram", "a dog", "a cat"], ["a guava", "an elephant"]).to(device)
with torch.no_grad():
    embedding = model.token_embedding(text)
# with torch.no_grad():
#     image_features = model.encode_image(image)
#     text_features = model.encode_text(text)
#     new_image_features = image_features.unsqueeze(dim=0)
#     new_text_features = text_features.unsqueeze(dim=0)
#     logits_per_image, logits_per_text = model(image, text)
#     probs = logits_per_image.softmax(dim=-1).cpu().numpy()

# print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

In [ ]:
embedding.shape

In [ ]:
image.shape, text.shape, text_features.shape, image_features.shape

In [ ]:
!pip install torchmetrics

In [ ]:
import torch
import torchmetrics


# Create target tensor with half 0s and half 1s
target_half_zeros = torch.zeros(540, 1920, dtype=torch.long)
target_half_ones = torch.ones(540, 1920, dtype=torch.long)
target = torch.cat((target_half_zeros, target_half_ones), dim=0)


pred = torch.zeros(1080, 1920, dtype=torch.long)
pred[:540, :960] = 1
pred[540:, 960:] = 1




In [ ]:
pred.shape, target.shape, pred, target

In [ ]:
accuracy = torchmetrics.classification.Accuracy(task="binary", num_classes=2, average="weighted")
accuracy(pred, target)

In [ ]:
import torchmetrics.functional as F
print(F.confusion_matrix(pred, target, task="binary", num_classes=2))
print(F.precision(pred, target, num_classes=2, task="binary",))
print(F.accuracy(pred, target, num_classes=2, task="binary",))

In [ ]:
jaccard = torchmetrics.classification.JaccardIndex(task="binary", num_classes=2)
jaccard(pred, target)

In [ ]:
import torchmetrics.functional as F
print(F.jaccard_index(pred, target, task="binary", num_classes=2))

In [ ]:
import torch
import torchmetrics.functional as F

preds = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
target = torch.tensor([0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1])

print(F.confusion_matrix(preds, target, task="binary", num_classes=2))
print(F.precision(preds, target, num_classes=2, task="binary",))
print(F.accuracy(preds, target, num_classes=2, task="binary",))

In [ ]:
jaccard = torchmetrics.classification.JaccardIndex(task="binary", num_classes=2, ignore_index=255, average="none")
jaccard(pred, target)

In [ ]:
jaccard = torchmetrics.classification.MulticlassJaccardIndex(num_classes=20, ignore_index=255, average="weighted")
jaccard(pred, target)

In [ ]:
jaccard = torchmetrics.classification.MulticlassJaccardIndex(num_classes=20, ignore_index=255, average="macro")
jaccard(pred, target)

In [ ]:
jaccard = torchmetrics.classification.MulticlassJaccardIndex(num_classes=20, ignore_index=255, average="micro")
jaccard(pred, target)